In [1]:
#primero que todo empezamos importando las librerias a usar
import pandas as pd
import numpy as np
from textblob import TextBlob

In [2]:
#vamos a leer todos los dataset a los cuales le estuvimos haciendo su respectiva limpieza y analisis
games_steam_df = pd.read_csv('C:/Users/CRISTIAN/Desktop/Proyecto final #  1/Proyecto-Final-individual-1-Henry/csv/games_steam_exportado_limpio.csv')
users_reviews_dfb = pd.read_csv('C:/Users/CRISTIAN/Desktop/Proyecto final #  1/Proyecto-Final-individual-1-Henry/csv/user_reviews_exportado_limpio.csv')
users_items_df = pd.read_csv('C:/Users/CRISTIAN/Desktop/Proyecto final #  1/Proyecto-Final-individual-1-Henry/csv/user_items_exportado_limpio.csv')

CREACION DE FUNCIONES QUE REQUERIMOS EN PARA EL MANEJO DE LA API 



Función Developer

def Developer(desarrollador: str): Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno: {"Año": 2023, "Cantidad de Items": 50, "Contenido Free": 27%}

In [6]:
#modificamos el nombre de la columna 'id', como 'item_id' para despues realizar uniones necesarias
games_steam_df.rename(columns={'id':'item_id'}, inplace=True)

In [7]:
#del dataframe 'games_steam' se extraen las columnas necesarias
developer_df = games_steam_df[['price', 'release_year', 'developer', 'item_id']]

In [9]:
#cambiamos el tipo de dato de la columna 'release_year'
developer_df['release_year'] = developer_df['release_year'].fillna(0).astype(int)

C:\Users\CRISTIAN\AppData\Local\Temp\ipykernel_19888\374144068.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  developer_df['release_year'] = developer_df['release_year'].fillna(0).astype(int)


In [10]:
#cambiamos el tipo de dato de la columna 'item_id'
developer_df['item_id'] = pd.to_numeric(developer_df['item_id'], errors= 'coerce').fillna(0).astype(int)

C:\Users\CRISTIAN\AppData\Local\Temp\ipykernel_19888\3733963460.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  developer_df['item_id'] = pd.to_numeric(developer_df['item_id'], errors= 'coerce').fillna(0).astype(int)


In [11]:
#procedemos a eliminar duplicados
developer_df.drop_duplicates(inplace=True)

C:\Users\CRISTIAN\AppData\Local\Temp\ipykernel_19888\1211942844.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  developer_df.drop_duplicates(inplace=True)


In [12]:
#creamos la funcion
def Developer(desarrollador):
    developer = developer_df[developer_df['developer'] == desarrollador]
    item_año = developer.groupby('release_year')['item_id'].count()
    gratis_año = developer[developer['price'] == 0.0].groupby('release_year')['item_id'].count()
    porcentaje_gratis = ((gratis_año / item_año) * 100).fillna(0).astype(int)
    return {
        'cantidad por año': gratis_año.to_dict(),
        'porcentaje gratuito por año': porcentaje_gratis.to_dict()
    }

In [22]:
#probamos
Developer('ProjectorGames')

{'cantidad por año': {2018: 1},
 'porcentaje gratuito por año': {2014: 0, 2015: 0, 2016: 0, 2018: 100}}

In [23]:
#exportamos
developer_df.to_parquet('C:/Users/CRISTIAN/Desktop/Proyecto final #  1/Proyecto-Final-individual-1-Henry/funciones_parquet/developer.parquet', index=False)

Función UserData

def UserData(user_id: str): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items. 

Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

In [24]:
#sacamos del dataframe 'users_items', las siguientes columnas
#items_count', 'user_id', 'item_id'.
items = users_items_df[['items_count', 'user_id', 'item_id']]

In [25]:
#sacamos del dataframe 'games_steam', las siguientes columnas
#'price', 'id'.
games = games_steam_df[['price', 'item_id']]

In [26]:
#borramos los duplicados 
games = games.drop_duplicates(subset= 'item_id',  keep='first')


In [28]:
#modificamos el tipo de dato de la columna 'item_id' a int
games['item_id'] = pd.to_numeric(games['item_id'], errors='coerce').fillna(0).astype('int')

In [29]:
#unimos ambos datas con las columnas que necesitamos
gasto_df = items.merge(games, on = 'item_id', how= 'left')

In [30]:
#remplazamos los datos nules por 0.0 ya que son los juegos que son gratiutos
gasto_df['price'] = gasto_df['price'].fillna(0.0)

In [31]:
#borramos la columan 'item_id'  ya que no la necesitamos mas
gasto_df = gasto_df.drop(columns= 'item_id')

In [32]:
#ahora agrupamos segun el usuario, la cantidad de dinero gastado
gasto_agr_df = gasto_df.groupby('user_id')['price'].sum().reset_index()

In [34]:
#creamos otro dataframe con los items por usuario
items_df = gasto_df[['user_id', 'items_count']]

In [35]:
#eliminamos los duplicado de id
items_df.drop_duplicates(subset= 'user_id', keep= 'first', inplace= True)

C:\Users\CRISTIAN\AppData\Local\Temp\ipykernel_19888\2949334071.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  items_df.drop_duplicates(subset= 'user_id', keep= 'first', inplace= True)


In [36]:
#paso final unimos los dataframe items por usuario 'items', con el dataframe total de dinero gastado por usuario 'gastos'
userdata_df = items_df.merge(gasto_agr_df, on = 'user_id', how= 'right')

In [37]:
users_reviews_dfb = users_reviews_dfb[['user_id', 'recommend']]

In [40]:
#creamos la funcion
def UserData(user_id):
    user = users_reviews_dfb[users_reviews_dfb['user_id'] == user_id]
    gasto_dinero = userdata_df[userdata_df['user_id'] == user_id]['price'].iloc[0]
    compra_items = userdata_df[userdata_df['user_id'] == user_id]['items_count'].iloc[0]

    recomendaciones = user['recommend'].sum()
    reviews = len(users_reviews_dfb['user_id'])
    recomendaciones_porcet = ((recomendaciones / reviews) * 100)
    return {
        'user_id': user_id,
        'dinero gastado': gasto_dinero,
        'items comprados': compra_items.astype(int),
        'Porcentaje de recomendaciones': round(recomendaciones_porcet, 3)
    }

In [41]:
#probamos
UserData('zzyfo')

{'user_id': 'zzyfo',
 'dinero gastado': 9.99,
 'items comprados': 84,
 'Porcentaje de recomendaciones': 0.0}

In [42]:
#exportamos
users_reviews_dfb.to_parquet('C:/Users/CRISTIAN/Desktop/Proyecto final #  1/Proyecto-Final-individual-1-Henry/funciones_parquet/users_reviews.parquet', index=False)
userdata_df.to_parquet('C:/Users/CRISTIAN/Desktop/Proyecto final #  1/Proyecto-Final-individual-1-Henry/funciones_parquet/userdata.parquet', index=False)

Funcion UserForGenre

def UserForGenre(genero: str): Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento. Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [6]:
#extraemos las columans necesarias de cada uno de los dataframe
genres = games_steam_df[['genres', 'release_year', 'item_id']]

In [7]:
genres

,genres,release_year,item_id
0,Action,2018.0,761140.0
1,Casual,2018.0,761140.0
2,Indie,2018.0,761140.0
3,Simulation,2018.0,761140.0
4,Strategy,2018.0,761140.0
...,...,...,...
74829,Racing,2018.0,610660.0
74830,Simulation,2018.0,610660.0
74831,Casual,2017.0,658870.0
74832,Indie,2017.0,658870.0


In [8]:
playtime = users_items_df[['user_id', 'playtime_forever', 'item_id']]

In [9]:
#modificamos el tipo de columna 'item_id', para unir los dataframe
genres['item_id'] = pd.to_numeric(genres['item_id'], errors='coerce').fillna(0).astype(int)

C:\Users\CRISTIAN\AppData\Local\Temp\ipykernel_12344\2118049754.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genres['item_id'] = pd.to_numeric(genres['item_id'], errors='coerce').fillna(0).astype(int)


In [10]:
userforgenre_df = genres.merge(playtime, on= 'item_id')

In [11]:
userforgenre_df.drop(columns = 'item_id', inplace=True)

In [12]:
userforgenre_df = userforgenre_df.dropna(subset= ['release_year'])

In [13]:
userforgenre_df['release_year'].unique()

array([1997., 1998., 2006., 2003., 2005., 2007., 2002., 2000., 1995.,
       1996., 1994., 2001., 1993., 2004., 1999., 2008., 2009., 1992.,
       1989., 2010., 2011., 2013., 2012., 2014., 1983., 1984., 2015.,
       2016., 2017., 1987., 1991., 1988., 1990.])

In [14]:
userforgenre_df['release_year'] = userforgenre_df['release_year'].astype(int)

In [15]:
#tomamos un subconjunto aleatorio del 5% de las filas de 'userforgenre_df'
#reteamos el indice limpio y continuo desde 0, y descarta el indice antiguo evitando
#que se convierta en una columna adicional en el dataframe.
userforgenre_df = userforgenre_df.sample(frac=0.05, random_state=5).reset_index(drop=True)

In [16]:
userforgenre_df.shape

(8796, 4)

In [17]:
#creamos la funcion
def UserForGenre(genre):
    df_genre = userforgenre_df[userforgenre_df['genres'] == genre]
    df_genre['playtime_forever'] = (df_genre['playtime_forever'] / 60 / 60).astype(int)
    playtime_user_max = df_genre.loc[df_genre['playtime_forever'].idxmax(), 'user_id']
    playtime_yearly = df_genre.groupby('release_year')['playtime_forever'].sum().reset_index()
    list_playtime = [{'año': int(year), 'Horas': int(hours)}for year, hours in zip(playtime_yearly['release_year'], playtime_yearly['playtime_forever'])]
    return {
        'usuario con mas horas jugadas para genero' + genre: playtime_user_max, 'Horas jugadas':  playtime_yearly
    }
    

In [22]:
#probamos
UserForGenre('Indie')

C:\Users\CRISTIAN\AppData\Local\Temp\ipykernel_12344\1686506141.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre['playtime_forever'] = (df_genre['playtime_forever'] / 60 / 60).astype(int)


{'usuario con mas horas jugadas para generoIndie': 'Slinkzxo',
 'Horas jugadas':     release_year  playtime_forever
 0           1988                 0
 1           1997                 0
 2           1999                 2
 3           2000                 0
 4           2001                 0
 5           2002                 0
 6           2003                 0
 7           2005                 0
 8           2006               189
 9           2007                 0
 10          2008                 2
 11          2009                 0
 12          2010                 0
 13          2011                72
 14          2012                26
 15          2013                33
 16          2014                 6
 17          2015                13
 18          2016                27
 19          2017                32}

In [23]:
#exportamos
userforgenre_df.to_parquet('C:/Users/CRISTIAN/Desktop/Proyecto final #  1/Proyecto-Final-individual-1-Henry/funciones_parquet/userforgenre.parquet', index=False)

Función BestDeveloperYear

def BestDeveloperYear(año : int): Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos) Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [24]:
#extaemos las columnas que necesitamos de cada dataframe
reviews = users_reviews_dfb[['item_id', 'recommend', 'sentiment']]


In [25]:
developer = games_steam_df[['item_id', 'release_year', 'developer']]

In [26]:
#convertimos la columan 'item_id' a tipo 'int'
developer['item_id'] = pd.to_numeric(developer['item_id'], errors='coerce').fillna(0).astype(int)

C:\Users\CRISTIAN\AppData\Local\Temp\ipykernel_12344\1118179134.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  developer['item_id'] = pd.to_numeric(developer['item_id'], errors='coerce').fillna(0).astype(int)


In [27]:
#unimos los dataframe obtenidos anteriormente
bestdeveloperyear_df = reviews.merge(developer, on = 'item_id').dropna()

In [29]:
#eliminamos los nulos de la columna 'release_year'
bestdeveloperyear_df.dropna(subset= ['release_year'])

,item_id,recommend,sentiment,release_year,developer
0,1250,True,2,2009.0,Tripwire Interactive
1,248820,True,2,2013.0,"Hopoo Games, LLC"
2,248820,True,2,2013.0,"Hopoo Games, LLC"
3,248820,True,2,2013.0,"Hopoo Games, LLC"
4,250320,True,2,2013.0,Telltale Games
...,...,...,...,...,...
55292,233270,True,2,2013.0,Ubisoft Montreal
55293,273110,True,2,2014.0,Nexon
55294,273110,True,2,2014.0,Nexon
55295,273110,True,2,2014.0,Nexon


In [30]:
#cambiamos el tipo de dato de float a int
bestdeveloperyear_df['release_year'] = bestdeveloperyear_df['release_year'].astype(int)

In [31]:
bestdeveloperyear_df.drop(columns= 'item_id', inplace = True)

In [32]:
#creamos la funcion
def BestDeveloperYear(year):
    año = bestdeveloperyear_df[bestdeveloperyear_df['release_year'] == year]
    filtro = año[(año['sentiment'] == 2) & (año['recommend'] == True)]
    developer = filtro.groupby('developer')['sentiment'].sum().reset_index()
    ordenado_df = developer.sort_values(by='sentiment', ascending=False)
    developers_top = ordenado_df.head(3)
    result = [{"Puesto {}: {}".format(i+1, row['developer'])} for i, (_, row) in enumerate(developers_top.iterrows())]
    return result

In [48]:
#Probamos
BestDeveloperYear(2006)

[{'Puesto 1: Facepunch Studios'},
 {'Puesto 2: Nabi Studios'},
 {'Puesto 3: Outerlight Ltd.'}]

In [49]:
bestdeveloperyear_df.to_parquet('C:/Users/CRISTIAN/Desktop/Proyecto final #  1/Proyecto-Final-individual-1-Henry/funciones_parquet/bestdeveloperyear.parquet', index=False)

Función DeveloperReviewsAnalysis

def developer_reviews_analysis( desarrolladora: str): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo. Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}

In [50]:
#volvemos y extraemos la columnas que necesitamos de cada dataframe
developer = games_steam_df[['developer', 'item_id']].dropna()


In [51]:
reviews = users_reviews_dfb[['item_id', 'sentiment']].dropna()

In [52]:
#volvemos a repetir el paso de cambiar el tipo de dato de la columna 'item_id' a int
developer['item_id'] = developer['item_id'].astype(int)

In [53]:
#unimos los dataframes 
reviews_developer_df = developer.merge(reviews, on= 'item_id')

In [54]:
#filtramos la columna 'sentiment', no teniento encuenta las reseñas que se encuentan categorizadas
#como neutras
reviews_developer_df = reviews_developer_df[reviews_developer_df['sentiment'] != 1]

In [55]:
#verificamos
reviews_developer_df['sentiment'].unique()

array([0, 2], dtype=int64)

In [56]:
#creamos la funcion
def DeveloperReviewsAnalysis(desarrollador):
    developer = reviews_developer_df[reviews_developer_df['developer'] == desarrollador]
    positivo = 0
    negativo = 0
    for sentiment in developer['sentiment']:
        if sentiment == 2:
            positivo += 1
        else: 
            negativo += 1
    return {
        desarrollador: [f"Negative = {negativo}", f"positive = {positivo}"]
    }

In [57]:
#probamos
DeveloperReviewsAnalysis('Valve')

{'Valve': ['Negative = 1072', 'positive = 4300']}

In [58]:
reviews_developer_df.to_parquet('C:/Users/CRISTIAN/Desktop/Proyecto final #  1/Proyecto-Final-individual-1-Henry/funciones_parquet/DeveloperReviewsAnalysis.parquet', index=False)